In [8]:
import torchani
import importlib
importlib.reload(torchani)
from ase.io import read, write
import torch
import sys
import ase
from ase.io import read, write
import matplotlib.pyplot as plt
from ase.units import fs
from copy import deepcopy
import torchani
import numpy as np

In [9]:
ani2x = torchani.models.ANI2x()
al = read("/Users/Cas/Work/HMD/PEG3/peg_all.xyz", ":")
atoms = al[0]

In [37]:
def get_comm_energy_forces(ani2x, at):
    Es = []
    Fs = []

    for i in range(8):
        calc = ani2x[i].ase()
        at.set_calculator(calc)
        Es.append(at.get_potential_energy())
        Fs.append(at.get_forces())

    meanE = np.mean(Es)
    meanF = np.mean(Fs, axis=0)

    varE = np.sum([ (Es[i] - meanE)**2 for i in range(8)])/8
    varF = np.sum([2 * (Es[i] - meanE) * (Fs[i] - meanF) for i in range(8)], axis=0)/8

In [38]:
get_comm_energy_forces(ani2x, al[1])

In [72]:
at = al[1]

In [100]:
species = torch.tensor(at.get_atomic_numbers(), dtype=torch.long, device=device)
species = species.unsqueeze(0)

In [101]:
coordinates = torch.tensor(at.get_positions(), requires_grad=True, device=device)

In [102]:
len(species)

1

In [103]:
len(coordinates)

31

In [108]:
model = torchani.models.ANI2x(periodic_table_index=True)

energy = model((species, coordinates)).energies

IndexError: index out of range in self

In [106]:
cell = torch.tensor(at.get_cell(complete=True))
pbc = torch.tensor(at.get_pbc(), dtype=torch.bool)
pbc_enabled = pbc.any().item()

In [109]:


model = torchani.models.ANI2x(periodic_table_index=True)

#energy = model((species, coordinates)).energies

coordinates = torch.tensor([[[0.03192167, 0.00638559, 0.01301679],
                             [-0.83140486, 0.39370209, -0.26395324],
                             [-0.66518241, -0.84461308, 0.20759389],
                             [0.45554739, 0.54289633, 0.81170881],
                             [0.66091919, -0.16799635, -0.91037834]]],
                           requires_grad=True, device=device)
# In periodic table, C = 6 and H = 1
species = torch.tensor([[6, 1, 1, 1, 1]], device=device)

###############################################################################
# Now let's compute energy and force:
energy = model((species, coordinates)).energies

In [68]:
species

tensor([[8, 6, 6, 8, 6, 6, 8, 6, 6, 8, 6, 6, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]])

In [3]:
calc = ani2x[1].ase()
at.set_calculator(calc)

In [44]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = torchani.models.ANI2x(periodic_table_index=True).to(device)

###############################################################################
# Now let's define the coordinate and species. If you just want to compute the
# energy and force for a single structure like in this example, you need to
# make the coordinate tensor has shape ``(1, Na, 3)`` and species has shape
# ``(1, Na)``, where ``Na`` is the number of atoms in the molecule, the
# preceding ``1`` in the shape is here to support batch processing like in
# training. If you have ``N`` different structures to compute, then make it
# ``N``.
#
# .. note:: The coordinates are in Angstrom, and the energies you get are in Hartree
coordinates = torch.tensor([[[0.03192167, 0.00638559, 0.01301679],
                             [-0.83140486, 0.39370209, -0.26395324],
                             [-0.66518241, -0.84461308, 0.20759389],
                             [0.45554739, 0.54289633, 0.81170881],
                             [0.66091919, -0.16799635, -0.91037834]]],
                           requires_grad=True, device=device)
# In periodic table, C = 6 and H = 1
species = torch.tensor([[6, 1, 1, 1, 1]], device=device)

###############################################################################
# Now let's compute energy and force:
energy = model((species, coordinates)).energies
derivative = torch.autograd.grad(energy.sum(), coordinates)[0]
force = -derivative

###############################################################################
# And print to see the result:
print('Energy:', energy.item())
print('Force:', force.squeeze())

###############################################################################
# you can also get the atomic energies (WARNING: these have no physical
# meaning) by calling:
_, atomic_energies = model[1].atomic_energies((species, coordinates))

/Users/Cas/miniconda3/lib/python3.8/site-packages/torchani-2.3.dev60+ga85e330.d20210518-py3.8.egg/torchani/resources/
Energy: -40.45979069605341
Force: tensor([[ 0.0478, -0.1304, -0.0551],
        [-0.1353,  0.1581, -0.0776],
        [ 0.0804, -0.0388,  0.0387],
        [ 0.0254,  0.0076,  0.0433],
        [-0.0183,  0.0035,  0.0508]])


In [47]:
atomic_energies

tensor([[-38.0564,  -0.5861,  -0.5958,  -0.6112,  -0.6099]],
       grad_fn=<AsStridedBackward>)

In [25]:
atomic_energies

tensor([[-1036.1987,   -15.8001,   -16.0767,   -16.4552,   -16.4363]],
       grad_fn=<MulBackward0>)

In [30]:
atoms.calculator(calc)

AttributeError: 'Atoms' object has no attribute 'calculator'

In [29]:
print(atoms.get_potential_energy())

-18816.786657402994


In [31]:
atomic_energies

tensor([[-1036.1987,   -15.8001,   -16.0767,   -16.4552,   -16.4363]],
       grad_fn=<MulBackward0>)

In [32]:
import numpy as np

In [35]:
np.sum(atomic_energies.detach().numpy())

-1100.9669

In [36]:
len(atoms)

31

In [38]:
_, atomic_energies = model.atomic_energies((species, coordinates), average=False)

In [39]:
atomic_energies 

tensor([[[-38.0841,  -0.5797,  -0.5898,  -0.6034,  -0.6027]],

        [[-38.0564,  -0.5861,  -0.5958,  -0.6112,  -0.6099]],

        [[-38.0744,  -0.5842,  -0.5930,  -0.6044,  -0.6037]],

        [[-38.0934,  -0.5769,  -0.5877,  -0.6012,  -0.6010]],

        [[-38.0904,  -0.5771,  -0.5880,  -0.6023,  -0.6016]],

        [[-38.0684,  -0.5833,  -0.5935,  -0.6079,  -0.6070]],

        [[-38.0826,  -0.5798,  -0.5901,  -0.6040,  -0.6033]],

        [[-38.0870,  -0.5781,  -0.5885,  -0.6033,  -0.6030]]],
       grad_fn=<AddBackward0>)

In [40]:
print('Atomic energies of first model, for species 6 1 1 1 1', atomic_energies[0, :, :])

Atomic energies of first model, for species 6 1 1 1 1 tensor([[-38.0841,  -0.5797,  -0.5898,  -0.6034,  -0.6027]],
       grad_fn=<SliceBackward>)
